In [2]:
import psycopg2
import numpy as np
import tensorflow as tf
import time
import paho.mqtt.client as mqtt
import json  # Thư viện để xử lý JSON
import joblib

# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model('fall_detection_model.h5')

# Tải scaler
scaler = joblib.load('scalerFall.pkl')

# Tải LabelEncoder
label_encoder = joblib.load('label_encoder.pkl')

# Thông tin HiveMQ Cloud
MQTT_HOST = "dd86dc72f53445328d345c2d91fd9470.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USER = "DoAnIoT"
MQTT_PASSWORD = "1234567890"
MQTT_TOPIC = "esp32/fall"

# Khởi tạo MQTT client
mqtt_client = mqtt.Client()
mqtt_client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
mqtt_client.tls_set()
mqtt_client.connect(MQTT_HOST, MQTT_PORT)

# Kết nối đến PostgreSQL
def get_data_from_db():
    conn = psycopg2.connect(
        host="localhost",
        database="db_DoAnIoT",
        user="postgres",
        password="110720"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM mpu6050_data ORDER BY timestamp DESC LIMIT 1")  # Lấy 1 dòng mới nhất
    data = cursor.fetchone()
    cursor.close()
    conn.close()
    return data

# Đọc và xử lý dữ liệu từ cơ sở dữ liệu
def process_and_predict():
    data = get_data_from_db()
    if data:
        print("Dữ liệu lấy từ PostgreSQL:", data)
        processed_data = np.array(data[:6], dtype=np.float32).reshape(1, -1)
        processed_data = scaler.transform(processed_data)
        processed_data = processed_data[..., np.newaxis]  # Dạng (1, 6, 1)

        predictions = model.predict(processed_data)
        predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])

        message = {
            "predicted_label": predicted_label[0]
        }
        mqtt_client.publish(MQTT_TOPIC, json.dumps(message))

        print("Nhãn dự đoán:", predicted_label[0])
        for label, prob in zip(label_encoder.classes_, predictions[0]):
            print(f"{label}: {prob:.2f}")
    else:
        print("Không có dữ liệu mới!")

# Vòng lặp liên tục, chạy mỗi 2 giây
while True:
    process_and_predict()
    time.sleep(2)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14900\1586932831.py:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


Dữ liệu lấy từ PostgreSQL: (1597, 0.5, 0.33, 9.9, -0.03, -0.01, -0.03, datetime.datetime(2024, 12, 18, 22, 26, 5, 375457, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200))))
1/1 [==============================] - 0s 346ms/step
Nhãn dự đoán: light
fall: 0.00
lfall: 0.00
light: 1.00
rfall: 0.00
sit: 0.00
step: 0.00
walk: 0.00
Dữ liệu lấy từ PostgreSQL: (1597, 0.5, 0.33, 9.9, -0.03, -0.01, -0.03, datetime.datetime(2024, 12, 18, 22, 26, 5, 375457, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200))))
1/1 [==============================] - 0s 47ms/step
Nhãn dự đoán: light
fall: 0.00
lfall: 0.00
light: 1.00
rfall: 0.00
sit: 0.00
step: 0.00
walk: 0.00
Dữ liệu lấy từ PostgreSQL: (1597, 0.5, 0.33, 9.9, -0.03, -0.01, -0.03, datetime.datetime(2024, 12, 18, 22, 26, 5, 375457, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200))))
1/1 [==============================] - 0s 50ms/step
Nhãn dự đoán: light
fall: 0.00
lfall: 0.00
light: 1.00
rfall: 0.00
sit: 0.00
step: 0.00
walk:

KeyboardInterrupt: 

In [3]:
import numpy as np
import tensorflow as tf
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model('fall_detection_model.h5')

# Tải scaler và label encoder
scaler = joblib.load('scalerFall.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Dữ liệu mẫu
sample_data = np.array([[1.16, 3.06, 9.11, 0.61, -0.83, 0.12]], dtype=np.float32)

# Chuẩn hóa dữ liệu
processed_data = scaler.transform(sample_data)
processed_data = processed_data[..., np.newaxis]  # Thêm trục kênh

# Dự đoán
predictions = model.predict(processed_data)
predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])

# In kết quả ra màn hình
print("Dữ liệu dự đoán:", sample_data)
print("Nhãn dự đoán:", predicted_label[0])
for label, prob in zip(label_encoder.classes_, predictions[0]):
    print(f"{label}: {prob:.2f}")


1/1 [==============================] - 0s 279ms/step
Dữ liệu dự đoán: [[ 1.16  3.06  9.11  0.61 -0.83  0.12]]
Nhãn dự đoán: light
fall: 0.16
lfall: 0.00
light: 0.67
rfall: 0.18
sit: 0.00
step: 0.00
walk: 0.00
